### ספריות

In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import Point

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


### העלת משתנים להרצת הקוד

In [3]:
path = os.getcwd()

parent = os.path.dirname(path)

software_data_folder_location = os.path.dirname(parent)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_data_folder_location))

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

sys.path.append(software_data_folder_location)

## פונקציות

In [4]:
from py_scripts.adding_students_to_the_population_and_age_distribution import adding_students_to_the_population_and_age_distribution
from py_scripts.adding_yeshiva_students_to_the_population_size import adding_yeshiva_students_to_the_population_size
from py_scripts.quantification_of_employees_in_jtmt_area import quantification_of_employees_in_jtmt_area
from py_scripts.unemployment_calculation import unemployment_calculation
from py_scripts.calculation_of_days import calculation_of_days

In [5]:
# def drop_geo(geoDF):
#     geoDF = geoDF.drop(columns='geometry')
#     return geoDF

In [6]:
# def double_taz_num(df):
#     dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
#     return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [7]:
# def make_point(df):
#     df_point=df.copy()
#     df_point['centroid'] = df_point.representative_point()
#     df_point=df_point.set_geometry('centroid')
#     df_point=df_point.drop(columns=['geometry'],axis=1)
#     return df_point

In [8]:
# def up_load_gdb(path,layer_name):
#     path='{}'.format(path)
#     layer_list=fiona.listlayers(path)
#     gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
#     return gpd_layer

In [9]:
# def up_load_shp(path):
#     path='{}'.format(path)
#     gpd_layer=gpd.read_file(path)
#     return gpd_layer

In [10]:
# def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
#     code_to_find=list(df_to_geoode[unique_field].unique())

#     gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

#     gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

#     gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

#     return gpd_for_geocode_in_one_taz

In [11]:
# def up_load_df(folder_path,file_name):
    
#     path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
#     df=pd.read_excel(path_df)
#     df=df.dropna(how='all')

#     return df


### פונקציות גלובליות

In [12]:
from functions import make_point
from functions import up_load_df
from functions import up_load_gdb
from functions import up_load_shp

In [13]:
#העלת משתנים להרצת הקוד
# df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

# software_data_folder_location=df_inputs_outputs['location'][0]

# forecast_version_folder_location=df_inputs_outputs['location'][1]


## להריץ תלמידים

In [14]:
%run "C:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\people\students_current_year_230706\students_current_year_230706.ipynb"
# %run '/people/students_current_year_230706/students_current_year_230706.ipynb'

C:\Users\dpere\AppData\Local\Temp\ipykernel_31796\284509738.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  student_yeshiva=student_yeshiva[col].pivot_table(index='Taz_num',aggfunc=sum)[[
C:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\people\students_current_year_230706\py_scripts\school.py:8: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taz['student_gov']=taz_student[['Taz_num','num_of_students']].pivot_table(index='Taz_num',aggfunc=sum)[['num_of_students']]
C:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\people\students_current_year_230706\py_scripts\school.py:14:

## להעלות שכבות

In [15]:
#העלת משתנים להרצת הקוד
# df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

# software_data_folder_location=df_inputs_outputs['location'][0]

# forecast_version_folder_location=df_inputs_outputs['location'][1]


In [16]:
path=r'{}\background_files\EMP_KIBOLET.gdb'.format(software_data_folder_location)
EMP_kibolet=up_load_gdb(path,'EMP_kibolet')

In [17]:
EMP_kibolet=EMP_kibolet.fillna(0)

In [18]:
EMP_kibolet['emp_without_palestin']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

In [19]:
emp_without_palestin_not_okev=EMP_kibolet['emp_without_palestin'].sum().item()

In [20]:
emp_Education=taz.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

In [21]:
student_dorms=up_load_shp(r'{}\background_files\student_dorms.shp'.format(software_data_folder_location))

In [22]:
# taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms)[['Taz_num','student_nu']].pivot_table(index='Taz_num',aggfunc=sum)

In [23]:
# taz=taz.fillna(0)

In [24]:
# taz['pop_without_dorms_yeshiva']=taz['pop']

In [25]:
# taz['pop']=taz['pop']+taz['student_dorms']

In [26]:
# taz['pop_20_just_from_aprt']=taz['pop_20']

In [27]:
# taz['pop_25_just_from_aprt']=taz['pop_25']

In [28]:
# taz['pop_20']=taz['pop_20']+taz['student_dorms']*0.6

In [29]:
# taz['pop_25']=taz['pop_25']+taz['student_dorms']*0.4

In [30]:
# col=['pop_0',
#  'pop_10',
#  'pop_15',
#  'pop_20',
#  'pop_25',
#  'pop_30',
#  'pop_35',
#  'pop_40',
#  'pop_45',
#  'pop_5',
#  'pop_50',
#  'pop_55',
#  'pop_60',
#  'pop_65',
#  'pop_70',
#  'pop_75up',]

# taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

# taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]
taz=adding_students_to_the_population_and_age_distribution(student_dorms, taz)

c:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\people\emp_current_year_230718\py_scripts\adding_students_to_the_population_and_age_distribution.py:4: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms)[['Taz_num','student_nu']].pivot_table(index='Taz_num',aggfunc=sum)


## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

In [31]:
taz=adding_yeshiva_students_to_the_population_size(taz)

## כימות מועסקים במרחב צתאל

In [32]:
taz=quantification_of_employees_in_jtmt_area(taz)

## חישוב אבטלה

In [33]:
taz=unemployment_calculation(taz)

c:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\people\emp_current_year_230718\py_scripts\unemployment_calculation.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1058.5125      925.0696     1368.9416     1401.4269      900.4801
  662.6458      925.0599      932.1215      931.33828983  997.1115
  567.256       832.8517     1120.5246      658.6882      798.3682
 1469.065      1668.1284      882.8552     1070.52809036  841.5235
  547.8269      625.63533771  421.00136291 1126.267       588.8191
  497.06827099  192.98137702  550.82128767  652.67722429    0.
  199.05712       0.         1029.46150669  958.554      1089.47169269
  258.27169331  316.24394151  868.35219351  381.68004435  346.99131233
 1150.0223      711.9509      926.2918      586.98056974  970.0056804
  242.39101412  485.02180886  448.62910236  700.57393582  187.79718023
  912.98938137  491.97901778    0.           31.82368444  3

## חישוב יוממות

In [34]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_data_folder_location))

In [35]:
taz=calculation_of_days(taz, commuting, emp_Education, emp_without_palestin_not_okev)

## פיזור עוקב

In [45]:
EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)

In [46]:
EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('kayim_emp>0')

In [47]:
EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

In [48]:
taz['taz_area']=taz.area

In [49]:
emp_buffer_taz=gpd.overlay(EMP_buffer,taz.reset_index())

In [50]:
emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

In [51]:
emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

In [52]:
taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()

In [53]:
taz['okev']=0
taz.loc[(~taz.index.isin(taz_num_no_need_okev))&(taz['pop']>0)&(taz['main_secto']!="Palestinia"),'okev']=1

In [54]:
taz.loc[taz['main_secto']=="arabs_behined_seperation_wall",'okev']=1

In [55]:
city_muni=['מודיעין עילית','בית שמש','ירושלים','מודיעין - מכבים - רעות']

## מקדם עוקב

In [56]:
okev_level=0.15

In [57]:
arab_city=taz.query('jew==0 & okev==1')['aprt_20'].sum().item()

In [58]:
arab_sub=0

In [59]:
ou_sub=taz.query('main_secto=="U_Orthodox" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [60]:
ou_city=taz.query('main_secto=="U_Orthodox" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [61]:
general_city=taz.query('main_secto=="Jewish" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [62]:
general_sub=taz.query('main_secto=="Jewish" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [63]:
base_okev_factor=(emp_okev-(okev_level*general_city-okev_level*(ou_sub+arab_city)-2*okev_level*arab_sub))/(general_sub+general_city+ou_city+ou_sub+arab_city+arab_sub)

NameError: name 'emp_okev' is not defined

In [ ]:
base_okev_factor=round(base_okev_factor,3)

In [ ]:
data = [['Jewish', base_okev_factor,base_okev_factor+okev_level],['arab', base_okev_factor-okev_level,base_okev_factor], ['U_Orthodox',base_okev_factor-2*okev_level,base_okev_factor-okev_level]]
 
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['sector', 'sub','city'])
okev_factor=df.set_index('sector')
okev_factor.to_excel(r'{}\Intermediates\okev_factor.xlsx'.format(software_data_folder_location))

In [ ]:
taz['emp_okev']=0

In [ ]:
taz.loc[(taz['jew']==0)&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['arab','city']

In [ ]:
taz.loc[(taz['main_secto']=="U_Orthodox")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','sub']

In [ ]:
taz.loc[(taz['main_secto']=="U_Orthodox")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','city']

In [ ]:
taz.loc[(taz['main_secto']=="Jewish")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','city']

In [ ]:
taz.loc[(taz['main_secto']=="Jewish")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','sub']

In [ ]:
taz.emp_okev.sum().item()-emp_okev

## פיצול שכבת לא עוקב לאזורי תנועה

In [ ]:
EMP_kibolet['emp_area']=EMP_kibolet.area

In [ ]:
EMP_kibolet['ID']=EMP_kibolet.index

In [ ]:
EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)

In [ ]:
EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

In [ ]:
EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

In [ ]:
EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

In [ ]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)').drop(columns='geometry_buff')

In [ ]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')

In [ ]:
EMP_kibolet_by_taz['id_area_for_pre']=EMP_kibolet_by_taz.reset_index().groupby(by='ID')['small_area'].sum()

In [ ]:
EMP_kibolet_by_taz[['id_area_for_pre']]

In [ ]:
EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']

In [ ]:
EMP_kibolet_by_taz['emp_without_palestin_in_taz']=EMP_kibolet_by_taz['emp_without_palestin']*EMP_kibolet_by_taz['prec_from_id']

In [ ]:
taz['emp_not_okev']=drop_geo(EMP_kibolet_by_taz).pivot_table(index='Taz_num',aggfunc=sum)[['emp_without_palestin_in_taz']]

In [ ]:
taz=taz.fillna(0)

In [ ]:
taz['total_emp']=taz['emp_not_okev']+taz['emp_okev']+taz['emp_Education']

In [ ]:
taz.query('main_secto!="Palestinian"')['total_emp'].sum()-emp_in_jtmt_area_without_mobile

## תוצאות

In [ ]:
taz.groupby(by='zonetype')[['total_emp']].sum()#.loc[['Jerusalem','Judea and Samaria']]

In [ ]:
taz.groupby(by='Muni_Heb')[['total_emp']].sum().loc[['ירושלים','בית שמש','ביתר עילית','מודיעין - מכבים - רעות','מעלה אדומים','מודיעין עילית']]

In [ ]:
taz.groupby(by=['Muni_Heb','main_secto'])[['total_emp']].sum().loc[['ירושלים']]

## התפלגות ענפי תעסוקה

In [ ]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_data_folder_location)).fillna(0)

In [ ]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [ ]:
emp_category_type[col]=emp_category_type[col]/100

In [ ]:
emp_category_type=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],emp_category_type).set_index('Taz_num')

In [ ]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [ ]:
taz.join(emp_category_type[col]).query('(emp_not_okev>0 | emp_okev>0 )& agri.isna() & main_secto!="Palestinian" ')

In [ ]:
taz=taz.join(emp_category_type[col]).fillna(0)

In [ ]:
for i in col:
    taz['{}'.format(i)]=taz['{}'.format(i)]*(taz['emp_not_okev']+taz['emp_okev'])

In [ ]:
taz['check_emp_sum']=taz[col].sum(axis=1)-(taz['emp_not_okev']+taz['emp_okev'])

In [ ]:
taz.query('check_emp_sum>10 |check_emp_sum<-10')

## הוספת סטודנטים אל כמות משקי בית

In [ ]:
taz=taz.fillna(0)

In [ ]:
taz['hh']=taz['aprt_20']+taz['student_dorms']/1.5

## הוספת תלמידי ישיבה (באזורים לא חרדיים) אל כמות אוכלוסיה,התפלגות גילים ומשקי בית (לכלל תלמידי הישיבה)

In [ ]:
taz['hh']=taz['hh']+taz['yeshiva_dorms_pop_sum']

In [ ]:
taz.loc[taz['main_secto']!="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

## הוספת אוכלוסיה פלסטינאית

In [ ]:
path=r'{}\background_files'.format(software_data_folder_location)
taz_demo_pls_2020=up_load_df(path,'taz_demo_pls_2020_and_pre_growth_till_2050')[['Taz_num','pop_2020']].set_index('Taz_num')

In [ ]:
col=['pop',
'pop_0',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_5',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'emp_Education',
'pop_emp_employed',
'total_emp',
'agri',
'Indus',
'Com_hotel',
'Business',
'Public',
'hh']

In [ ]:
taz.loc[taz['main_secto']=='Palestinian',col]=0

In [ ]:
taz.loc[taz['main_secto']=='Palestinian','pop']=taz['pop']+taz_demo_pls_2020['pop_2020']

## ייצוא לבקרה

In [ ]:
taz=taz.reset_index()

In [ ]:
pre_yeshiva_from_20_30_age=taz[['student_yeshiva_with_add_from_old']].sum().sum()/taz.query('jew==1')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [ ]:
pre_yeshiva_from_20_30_age

In [ ]:
taz.main_secto.unique()

In [ ]:
pre_uni_from_20_30_age=taz[['uni_students']].sum().sum()/taz.query('main_secto!="Palestinian"')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [ ]:
pre_uni_from_20_30_age

In [ ]:
list(taz)

In [ ]:
col=['Taz_num','Agg_taz_nu','Taz_name','Muni_Heb','jeru_metro',
 'zonetype',
 'in_jerusal','main_secto','aprt_20','student_dorms','student_yeshiva_with_add_from_old','hh','pop_without_dorms_yeshiva',
 'pop', 'hh_size','pop_emp_employed',
 'commuting','student_toddlers',
 'student_gov','cbs_muni_student_left_by_pre_of_demand_left','cbs_muni_student_left_by_pre_of_demand','student_chardi_not_gov',
 'student_arab_not_gov','student','uni_students',
 'student_yeshiva','yeshiva_dorms_pop_15',
 'yeshiva_dorms_pop_20',
 'yeshiva_dorms_pop_25',
 'yeshiva_dorms_pop_sum',
 'add_from_old_student_yeshiva',
 'add_from_kollim_demand', 'emp_from_uni_student','emp_from_Yeshiva_student','emp_from_student',
 'emp_Education','okev',
 'emp_okev',
 'emp_not_okev','total_emp',
 'agri',
 'Indus',
 'Com_hotel',
 'Business',
 'Public']

In [ ]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [ ]:
taz[col].to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## ייצוא בפורמט של מודלים

In [ ]:
taz['yosh']=0

In [ ]:
taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

In [ ]:
taz['jerusalem_city']=0

In [ ]:
taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

In [ ]:
taz=taz.reset_index()

In [ ]:
col=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'pop_emp_employed']

In [ ]:
col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [ ]:
taz=taz[col]

In [ ]:
taz.columns=col_new_name

In [ ]:
taz.to_excel(r'{}\forecast_2020_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## בקרות לוגיות

In [ ]:
taz.loc[taz['TAZ']<=0]

In [ ]:
taz.yosh.unique()

In [ ]:
taz.in_jerusalem_metropolin.unique()

In [ ]:
taz.jerusalem_city.unique()

In [ ]:
taz.sector.unique()

In [ ]:
col=['hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]<0]['TAZ'])

In [ ]:
taz_to_check

In [ ]:
taz.loc[~(taz['hh_total']<=taz['pop'])]

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['hh_total']>0].loc[taz['pop']<=0]

In [ ]:
col=['age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0].query('TAZ==511')#.to_excel('delet.xlsx')#[col].sum(axis=1)

In [ ]:
len(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0])

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop']>0].loc[taz['hh_total']<=0]

In [ ]:
col=[
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up']

In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['pop']<=0]['TAZ'])
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['hh_total']<=0]['TAZ'])

In [ ]:
taz_to_check

In [ ]:
col=['indus',
'com_hotel',
'business',
'public',
'education',
'agri']

In [ ]:
taz['emp_check']=round(taz[col].sum(axis=1)-taz['emp_tot'])

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['emp_check']!=0]

In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['emp_tot']<=0]['TAZ'])

In [ ]:
taz_to_check

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_emp_employed']>0].loc[taz['pop']<=0]